# Carga inicial de los datos de encuentas

Este notebook contiene un proceso de limpieza sobre los datos recolectados para el proyecto MIT LIFT LAB.

## 0. Carga de los datos

In [1]:
import pandas as pd
import re

In [2]:
df_raw = pd.read_csv('../data/raw/LIFT Digitization and Performance Index - Argentina - Fall 2025_October 29, 2025_08.44.csv', 
                     skiprows=[0, 2], 
                     encoding='utf-8')
df_raw.head()

,Start Date,End Date,Response Type,IP Address,Progress,Duration (in seconds),Finished,Recorded Date,Response ID,Recipient Last Name,...,Horario de apertura,Horario de cierre,¿Cuál fue el consumo energético del negocio el mes pasado en Kw?,Sexo de la persona encuestada,WhatsApp del la persona entrevistada (Recomendado para la siguiente fase y entrega de reporte final),¿El local de su negocio es rentado o propio?,¿Cuál es el máximo grado de educación de la persona entrevistada?,¿Cuál es la edad de la persona entrevistada?,lat,long
0,2025-10-02 13:38:55,2025-10-02 14:27:02,IP Address,*******,100,2887,True,2025-10-02 14:27:06,R_1IIdCNJ8zwPVCZb,*******,...,9:00 am,8:30 pm,NaN,Mujer,77124645,Propio,Primaria,Más de 60,NaN,NaN
1,2025-10-04 06:57:48,2025-10-04 07:46:01,IP Address,*******,100,2892,True,2025-10-04 07:46:05,R_6nx4McRsbAx4Hu7,*******,...,8:30 am,8:30 pm,1012,Mujer,1150081170,Propio,Universidad y más,De 51 a 60,NaN,NaN
2,2025-10-04 07:50:56,2025-10-04 08:06:45,IP Address,*******,100,948,True,2025-10-04 08:06:49,R_1fHu9kpBWusHc0C,*******,...,9:30 am,7:30 pm,no sabe,Hombre,1132652189,Rentado,Universidad y más,Más de 60,NaN,NaN
3,2025-10-04 08:08:52,2025-10-04 08:23:47,IP Address,*******,100,894,True,2025-10-04 08:23:48,R_3m2UzzPCRooCx2I,*******,...,9:00 am,8:00 pm,No sabe,Mujer,1126320744,Propio,Secundaria,De 51 a 60,NaN,NaN
4,2025-10-04 08:28:57,2025-10-04 08:44:38,IP Address,*******,100,940,True,2025-10-04 08:44:41,R_7zvbUlEW6KNAeOH,*******,...,8:30 am,9:00 pm,No lo sabe,Hombre,1126651956,Rentado,Secundaria,De 31 a 40,NaN,NaN


In [3]:
df_raw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 923 entries, 0 to 922
Data columns (total 66 columns):
 #   Column                                                                                                                                                              Non-Null Count  Dtype  
---  ------                                                                                                                                                              --------------  -----  
 0   Start Date                                                                                                                                                          923 non-null    object 
 1   End Date                                                                                                                                                            923 non-null    object 
 2   Response Type                                                                                                         

## 1. Eliminación de columnas innecesarias para el análisis

En este conjunto de datos "crudo" que cargamos hay muchas columnas de metadata/técnicas de Qualtrics que no aportan al análisis y por lo tanto se deben eliminar para así reducir el ruido. Luego hay otras columnas que se deberán eliminar posteriormente por un tema de que tampoco añaden al análisis, como lo puede ser el nombre del equipo, del encuestador o del negocio. Sin embargo no se hace esa eliminación ahora mismo porque vamos a necesitar posteriormente dicha información para cruzar los datos con nuevas encuestas entrantes sobre los mismos negocios, por lo tanto al final de esta notebook se procederá a guardar dos versiones del conjunto de datos procesado: una con las columnas mencionadas anteriormente, hecha para cruzar datos, y otra sin dichas columnas para poder realizar un análisis con las columnas que realmente resultan interesantes.

Sin más se procede a eliminar las columnas de metadata:

In [4]:
columnas_eliminar = [
    'Start Date', 'End Date', 'Response Type', 'IP Address', 
    'Progress', 'Duration (in seconds)', 'Finished', 'Recorded Date',
    'Response ID', 'Recipient Last Name', 'Recipient First Name', 
    'Recipient Email', 'External Data Reference', 
    'Location Latitude', 'Location Longitude',  # Estas están duplicadas en otras columnas
    'Distribution Channel', 'User Language',
    'Foto de la fachada del negocio - Id',
    'Foto de la fachada del negocio - Name',
    'Foto de la fachada del negocio - Size',
    'Foto de la fachada del negocio - Type',
    'Foto de la fachada del negocio - URL'  # Mantendríamos si se necesitan las fotos
]
columnas_a_eliminar = [col for col in columnas_eliminar if col in df_raw.columns]
df_clean = df_raw.drop(columns=columnas_a_eliminar)
print(f"Columnas eliminadas: {len(columnas_a_eliminar)}")
print(f"Dimensiones después de eliminar metadata: {df_clean.shape}")

Columnas eliminadas: 22
Dimensiones después de eliminar metadata: (923, 44)


## 2. Renombrar columnas con nombres más cortos y estandarizados

Otra cuestión con este conjunto de datos es que al provenir de una encuesta, sus columnas muestran a las preguntas hechas tal cual durante el proceso de recopilación. Algo que puede ayudar a reducir la carga cognitiva es acortar los nombres:

In [5]:
renombres = {
    'Nombre del equipo': 'equipo',
    'Nombre del comercio': 'comercio',
    'Nombre completo del Encuestador o Encuestadora': 'encuestador',
    'Coordenadas del negocio (Latitud, Longitud)': 'coordenadas_texto',
    'Tipo de negocio': 'tipo_negocio',
    '¿El negocio tiene reja para entrar? (¿los consumidores pueden entrar a la tienda para escoger y comprar los productos?)': 'tiene_reja',
    '¿Cuál fue el año de apertura del negocio?': 'anio_apertura',
    '¿Cuántas personas trabajaron en este negocio (con o sin paga) durante el mes anterior?': 'trabajadores_total',
    '¿Cuántas personas trabajaron con un salario fijo durante el último mes?': 'trabajadores_salario_fijo',
    'En tres meses, ¿esperas que el número total de trabajadores del negocio aumente, disminuya o permanezca igual?': 'expectativa_trabajadores_3m',
    '¿Tus ventas actuales están mejor, peor o igual en comparación con las de hace un mes?': 'ventas_vs_mes_anterior',
    '¿Por qué piensas que las ventas incrementaron/disminuyeron/permanecieron igual? (opcional)': 'razon_cambio_ventas',
    '¿Esperas que las ventas de tu negocio en tres meses sean mayores, menores o iguales?': 'expectativa_ventas_3m',
    '¿Durante el último mes tu nivel de inventario aumentó, disminuyó o permaneció igual?': 'cambio_inventario',
    '¿Esperas que dentro de tres meses el nivel de inventario incremente, disminuya o permanezca igual?': 'expectativa_inventario_3m',
    'Actualmente, ¿sientes que tu nivel de inventario es demasiado grande, muy poco o está en su nivel correcto?': 'percepcion_inventario',
    'En promedio, ¿el precio de venta de los productos que vende este establecimiento aumentó, bajó o permaneció igual a los precios de hace un mes?': 'cambio_precios',
    '¿Dentro de tres meses, esperas que los precios de los productos que vendes aumenten, disminuyan o permanezcan igual?': 'expectativa_precios_3m',
    'Qué tanto impacta a tu negocio: - El crimen': 'impacto_crimen',
    'Qué tanto impacta a tu negocio: - La falta de crédito': 'impacto_falta_credito',
    'Qué tanto impacta a tu negocio: - El incremento en el precio de los bienes': 'impacto_precio_bienes',
    'Qué tanto impacta a tu negocio: - La competencia': 'impacto_competencia',
    '¿Recibiste crédito para operar tu negocio de alguna de las siguientes entidades? - Bancos': 'credito_bancos',
    '¿Recibiste crédito para operar tu negocio de alguna de las siguientes entidades? - Proveedores': 'credito_proveedores',
    '¿Recibiste crédito para operar tu negocio de alguna de las siguientes entidades? - Familia': 'credito_familia',
    '¿Recibiste crédito para operar tu negocio de alguna de las siguientes entidades? - Gobierno': 'credito_gobierno',
    '¿Recibiste crédito para operar tu negocio de alguna de las siguientes entidades? - Crédito privado (conocidos)': 'credito_privado',
    'Opcional - ¿De alguna otra entidad recibiste crédito para operar tu negocio? Escribe cuál': 'credito_otro',
    '¿Cuál es la razón principal por la que tu negocio no tiene un crédito bancario? (Selecciona una)': 'razon_sin_credito',
    'Asume que tienes los medios para hacer crecer tu negocio, ¿quisieras que tu negocio creciera? (por ejemplo, con más empleados, más productos, un local más grande)': 'quiere_crecer',
    '¿Cuál es la razón principal por la que no quieres que tu negocio crezca? (Selecciona una)': 'razon_no_crecer',
    '¿Cuál sería el salario mensual más bajo que estaría dispuesto a aceptar para cerrar el negocio y\naceptar un trabajo hoy? (En pesos argentinos por mes)': 'salario_dejar_negocio',
    '¿Cómo describirías el nivel actual de adopción de tecnología digital en su negocio? (Seleccione una)': 'nivel_digitalizacion',
    '¿Cuáles de las siguientes tecnologías digitales utilizas actualmente en la operación del negocio? (Marque todas las que correspondan)': 'tecnologias_digitales',
    'Horario de apertura': 'horario_apertura',
    'Horario de cierre': 'horario_cierre',
    '¿Cuál fue el consumo energético del negocio el mes pasado en Kw?': 'consumo_energia_kw',
    'Sexo de la persona encuestada': 'sexo',
    'WhatsApp del la persona entrevistada (Recomendado para la siguiente fase y entrega de reporte final)': 'whatsapp',
    '¿El local de su negocio es rentado o propio?': 'tipo_local',
    '¿Cuál es el máximo grado de educación de la persona entrevistada?': 'educacion',
    '¿Cuál es la edad de la persona entrevistada?': 'rango_edad',
}
df_clean = df_clean.rename(columns=renombres)
print(f"Columnas renombradas: {len(renombres)}")

Columnas renombradas: 42


In [6]:
df_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 923 entries, 0 to 922
Data columns (total 44 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   equipo                       905 non-null    object 
 1   comercio                     738 non-null    object 
 2   encuestador                  922 non-null    object 
 3   coordenadas_texto            877 non-null    object 
 4   tipo_negocio                 923 non-null    object 
 5   tiene_reja                   923 non-null    object 
 6   anio_apertura                853 non-null    float64
 7   trabajadores_total           853 non-null    float64
 8   trabajadores_salario_fijo    853 non-null    float64
 9   expectativa_trabajadores_3m  853 non-null    object 
 10  ventas_vs_mes_anterior       851 non-null    object 
 11  razon_cambio_ventas          703 non-null    object 
 12  expectativa_ventas_3m        851 non-null    object 
 13  cambio_inventario   

## 3. Estandarizar formatos de texto

In [7]:
# Función para limpiar texto
def limpiar_texto(texto):
    if pd.isna(texto):
        return texto
    texto = str(texto).strip()
    # Estandarizar respuestas comunes
    texto = texto.replace('Sí', 'Si').replace('sí', 'Si')
    texto = texto.replace('  ', ' ')  # Quitar espacios dobles
    return texto

# Aplicar limpieza a columnas de texto
columnas_texto = df_clean.select_dtypes(include=['object']).columns
for col in columnas_texto:
    df_clean[col] = df_clean[col].apply(limpiar_texto)

print("Texto estandarizado en todas las columnas de tipo texto")

Texto estandarizado en todas las columnas de tipo texto


## 4. Convertir coordenadas a columnas numéricas separadas

In [8]:
if 'coordenadas_texto' in df_clean.columns:
    # Extraer lat y long de la columna de coordenadas
    coords_split = df_clean['coordenadas_texto'].str.extract(r'([+-]?\d+\.?\d*),\s*([+-]?\d+\.?\d*)')
    df_clean['latitud'] = pd.to_numeric(coords_split[0], errors='coerce')
    df_clean['longitud'] = pd.to_numeric(coords_split[1], errors='coerce')
    
    # Si hay columnas lat/long adicionales, usarlas como respaldo
    if 'lat' in df_clean.columns:
        df_clean['latitud'] = df_clean['latitud'].fillna(pd.to_numeric(df_clean['lat'], errors='coerce'))
    if 'long' in df_clean.columns:
        df_clean['longitud'] = df_clean['longitud'].fillna(pd.to_numeric(df_clean['long'], errors='coerce'))
    
    # Eliminar columnas redundantes
    cols_a_eliminar = ['coordenadas_texto', 'lat', 'long']
    df_clean = df_clean.drop(columns=[col for col in cols_a_eliminar if col in df_clean.columns])
    
    print(f"Coordenadas procesadas. Registros con coordenadas válidas: {df_clean['latitud'].notna().sum()}")

Coordenadas procesadas. Registros con coordenadas válidas: 814


## 5. Convertir respuestas Si/No a booleanos

In [9]:
columnas_binarias = [
    'tiene_reja', 'credito_bancos', 'credito_proveedores', 
    'credito_familia', 'credito_gobierno', 'credito_privado', 'quiere_crecer'
]
for col in columnas_binarias:
    if col in df_clean.columns:
        df_clean[col] = df_clean[col].map({'Si': True, 'No': False, 'Sí': True})

print(f"Columnas binarias convertidas: {[col for col in columnas_binarias if col in df_clean.columns]}")

Columnas binarias convertidas: ['tiene_reja', 'credito_bancos', 'credito_proveedores', 'credito_familia', 'credito_gobierno', 'credito_privado', 'quiere_crecer']


## 6. Convertir columnas numéricas

In [10]:
df_clean['salario_dejar_negocio']

0                                         0
1            no estaria dispuesta a dejarlo
2                                  13000000
3      Por un aumento sustantivo de salario
4                              No aceptaria
                       ...                 
918                                     NaN
919                                     NaN
920                                     NaN
921                                     NaN
922                                     NaN
Name: salario_dejar_negocio, Length: 923, dtype: object

In [11]:
def identificar_texto_en_num(df, columna):
    """
    Identifica si una columna numérica tiene valores de texto mezclados.
    Retorna información sobre la presencia de texto sin modificar el DataFrame.
    """
    if columna not in df.columns:
        return None
    
    # Intentar convertir a numérico
    valores_numericos = pd.to_numeric(df[columna], errors='coerce')
    
    # Identificar registros con texto
    mask_texto = df[columna].notna() & valores_numericos.isna()
    
    cantidad_numericos = valores_numericos.notna().sum()
    cantidad_texto = mask_texto.sum()
    
    # Obtener ejemplos de texto
    ejemplos_texto = df.loc[mask_texto, columna].head(10).tolist() if cantidad_texto > 0 else []
    
    return {
        'columna': columna,
        'tiene_texto': cantidad_texto > 0,
        'cantidad_numericos': cantidad_numericos,
        'cantidad_texto': cantidad_texto,
        'ejemplos_texto': ejemplos_texto
    }


def separar_text_num_salario_dejar_negocio(df):
    """
    Separa la columna salario_dejar_negocio en versiones numérica y de texto.
    Aplica reglas de negocio específicas para salarios argentinos.
    """
    columna = 'salario_dejar_negocio'
    
    if columna not in df.columns:
        return df
    
    col_num = f"{columna}_num"
    col_text = f"{columna}_text"
    
    # Inicializar columnas
    df[col_num] = None
    df[col_text] = None
    
    for idx, valor in df[columna].items():
        if pd.isna(valor):
            continue
            
        valor_str = str(valor).strip()
        
        # Intentar conversión directa a numérico
        try:
            valor_numeric = pd.to_numeric(valor_str.replace('.', '').replace(',', '.'))
            df.loc[idx, col_num] = valor_numeric
            continue
        except:
            pass
        
        # Es texto, guardar en columna de texto
        df.loc[idx, col_text] = valor_str
        
        # Intentar extraer número con reglas de negocio
        valor_lower = valor_str.lower()
        
        # Buscar la palabra "millones" o "millon"
        tiene_millones = 'millon' in valor_lower
        
        # Extraer todos los números del texto
        # Patrón que captura números con separadores (puntos o comas)
        numeros_encontrados = re.findall(r'\d+[.,\d]*', valor_str)
        
        if numeros_encontrados:
            # Tomar el primer/mayor número encontrado
            numero_str = numeros_encontrados[0]
            
            # Contar cuántos separadores tiene (puntos o comas)
            num_separadores = numero_str.count('.') + numero_str.count(',')
            
            # Limpiar el número: eliminar puntos y reemplazar comas por puntos
            numero_limpio = numero_str.replace('.', '').replace(',', '.')
            
            try:
                numero = float(numero_limpio)
                
                # Aplicar reglas de negocio
                if tiene_millones:
                    # Si dice "millones", multiplicar por 1.000.000
                    numero = numero * 1_000_000
                elif num_separadores == 2:
                    # Si tiene dos separadores, es formato x.xxx.xxx (millones)
                    # El número ya está bien formateado al quitar los puntos
                    pass
                elif num_separadores == 1 and '.' in numero_str:
                    # Si tiene un punto, es separador de miles (x.xxx)
                    # Ya se eliminó el punto, así que está correcto
                    pass
                
                df.loc[idx, col_num] = numero
            except:
                pass
    
    # Eliminar columna original
    df = df.drop(columns=[columna])
    
    return df


def separar_text_num_consumo_energia_kw(df):
    """
    Separa la columna consumo_energia_kw en versiones numérica, de texto y cash.
    Aplica reglas de negocio específicas para consumo energético en kW.
    
    Regla de negocio para consumo razonable:
    - Un local comercial típico en Argentina consume entre 200-5000 kWh/mes
    - Locales grandes (cosa que no entra en esta encuesta) pueden llegar hasta 10.000 kWh/mes
    - Todo valor > 10.000 kW se considera probablemente dinero (pesos argentinos)
    """
    columna = 'consumo_energia_kw'
    
    if columna not in df.columns:
        return df
    
    col_num = f"{columna}_num"
    col_text = f"{columna}_text"
    col_cash = f"{columna}_cash"
    
    # Umbral máximo razonable de consumo en kW para un local comercial
    MAX_KW_RAZONABLE = 10000
    
    # Inicializar columnas
    df[col_num] = None
    df[col_text] = None
    df[col_cash] = None
    
    for idx, valor in df[columna].items():
        if pd.isna(valor):
            continue
            
        valor_str = str(valor).strip()
        
        # Intentar conversión directa a numérico
        try:
            valor_numeric = pd.to_numeric(valor_str.replace('.', '').replace(',', '.'))
            
            # Verificar si es un valor razonable de kW o si es dinero
            if valor_numeric > MAX_KW_RAZONABLE:
                # Probablemente es dinero (pesos argentinos)
                df.loc[idx, col_cash] = valor_numeric
                df.loc[idx, col_text] = valor_str  # También guardar el texto original
            else:
                # Es un consumo razonable en kW
                df.loc[idx, col_num] = valor_numeric
            continue
        except:
            pass
        
        # Es texto, guardar en columna de texto
        df.loc[idx, col_text] = valor_str
        
        # Intentar extraer número
        numeros_encontrados = re.findall(r'\d+[.,\d]*', valor_str)
        
        if numeros_encontrados:
            numero_str = numeros_encontrados[0]
            
            # Para consumo energético, aplicar misma lógica de limpieza
            num_separadores = numero_str.count('.') + numero_str.count(',')
            numero_limpio = numero_str.replace('.', '').replace(',', '.')
            
            try:
                numero = float(numero_limpio)
                
                # Verificar si es razonable o es dinero
                if numero > MAX_KW_RAZONABLE:
                    df.loc[idx, col_cash] = numero
                else:
                    df.loc[idx, col_num] = numero
            except:
                pass
    
    # Eliminar columna original
    df = df.drop(columns=[columna])
    
    return df


# Columnas que deberían ser numéricas
columnas_numericas = [
    'anio_apertura', 'trabajadores_total', 'trabajadores_salario_fijo', 
    'salario_dejar_negocio', 'consumo_energia_kw'
]

print("="*80)
print("IDENTIFICACIÓN DE TEXTO EN COLUMNAS NUMÉRICAS")
print("="*80)

columnas_con_texto = []

for col in columnas_numericas:
    if col in df_clean.columns:
        info = identificar_texto_en_num(df_clean, col)
        
        if info and info['tiene_texto']:
            columnas_con_texto.append(col)
            print(f"\n{col}:")
            print(f"   - Valores numéricos: {info['cantidad_numericos']}")
            print(f"   - Valores con texto: {info['cantidad_texto']}")
            print(f"   - Ejemplos de texto:")
            for ejemplo in info['ejemplos_texto'][:5]:
                print(f"      • '{ejemplo}'")
        else:
            # Convertir directamente a numérico si no hay texto
            df_clean[col] = pd.to_numeric(df_clean[col], errors='coerce')
            print(f"\n{col}: Convertida a numérico (sin texto)")

print("\n" + "="*80)
print(f"Columnas con texto identificadas: {columnas_con_texto}")
print("="*80)


IDENTIFICACIÓN DE TEXTO EN COLUMNAS NUMÉRICAS

anio_apertura: Convertida a numérico (sin texto)

trabajadores_total: Convertida a numérico (sin texto)

trabajadores_salario_fijo: Convertida a numérico (sin texto)

salario_dejar_negocio:
   - Valores numéricos: 343
   - Valores con texto: 435
   - Ejemplos de texto:
      • 'no estaria dispuesta a dejarlo'
      • 'Por un aumento sustantivo de salario'
      • 'No aceptaria'
      • 'No está dispuesto a cerrar'
      • '4.000.000'

consumo_energia_kw:
   - Valores numéricos: 268
   - Valores con texto: 405
   - Ejemplos de texto:
      • 'no sabe'
      • 'No sabe'
      • 'No lo sabe'
      • 'No sabe'
      • 'No'

Columnas con texto identificadas: ['salario_dejar_negocio', 'consumo_energia_kw']


In [12]:
print("\n" + "="*80)
print("APLICANDO SEPARACIÓN CON REGLAS DE NEGOCIO")
print("="*80)

if 'salario_dejar_negocio' in columnas_con_texto:
    print("\nProcesando 'salario_dejar_negocio'...")
    df_clean = separar_text_num_salario_dejar_negocio(df_clean)
    
    valores_num = df_clean['salario_dejar_negocio_num'].notna().sum()
    valores_text = df_clean['salario_dejar_negocio_text'].notna().sum()
    
    print(f"   - Valores numéricos finales: {valores_num}")
    print(f"   - Valores solo texto (sin número extraíble): {valores_text}")
    
    # Mostrar estadísticas de los valores numéricos
    if valores_num > 0:
        print(f"\n   Estadísticas de salario_dejar_negocio_num:")
        print(f"      - Mínimo: ${df_clean['salario_dejar_negocio_num'].min():,.0f}")
        print(f"      - Máximo: ${df_clean['salario_dejar_negocio_num'].max():,.0f}")
        print(f"      - Promedio: ${df_clean['salario_dejar_negocio_num'].mean():,.0f}")
        print(f"      - Mediana: ${df_clean['salario_dejar_negocio_num'].median():,.0f}")

if 'consumo_energia_kw' in columnas_con_texto:
    print("\n Procesando consumo_energia_kw...")
    df_clean = separar_text_num_consumo_energia_kw(df_clean)
    
    valores_num = df_clean['consumo_energia_kw_num'].notna().sum()
    valores_text = df_clean['consumo_energia_kw_text'].notna().sum()
    valores_cash = df_clean['consumo_energia_kw_cash'].notna().sum()
    
    print(f"   ✓ Valores en kW (razonables): {valores_num}")
    print(f"   ✓ Valores en pesos (> 10.000): {valores_cash}")
    print(f"   ✓ Valores solo texto (sin número extraíble): {valores_text}")
    
    # Mostrar estadísticas de los valores numéricos en kW
    if valores_num > 0:
        print(f"\n   Estadísticas de consumo_energia_kw_num (kW):")
        print(f"      - Mínimo: {df_clean['consumo_energia_kw_num'].min():,.0f} kW")
        print(f"      - Máximo: {df_clean['consumo_energia_kw_num'].max():,.0f} kW")
        print(f"      - Promedio: {df_clean['consumo_energia_kw_num'].mean():,.0f} kW")
        print(f"      - Mediana: {df_clean['consumo_energia_kw_num'].median():,.0f} kW")
    
    # Mostrar estadísticas de los valores en pesos
    if valores_cash > 0:
        print(f"\n   Estadísticas de consumo_energia_kw_cash (pesos):")
        print(f"      - Mínimo: ${df_clean['consumo_energia_kw_cash'].min():,.0f}")
        print(f"      - Máximo: ${df_clean['consumo_energia_kw_cash'].max():,.0f}")
        print(f"      - Promedio: ${df_clean['consumo_energia_kw_cash'].mean():,.0f}")
        print(f"      - Mediana: ${df_clean['consumo_energia_kw_cash'].median():,.0f}")

print("\n" + "="*80)


APLICANDO SEPARACIÓN CON REGLAS DE NEGOCIO

Procesando 'salario_dejar_negocio'...
   - Valores numéricos finales: 491
   - Valores solo texto (sin número extraíble): 353

   Estadísticas de salario_dejar_negocio_num:
      - Mínimo: $0
      - Máximo: $90,000,000
      - Promedio: $2,991,666
      - Mediana: $2,000,000

 Procesando consumo_energia_kw...
   ✓ Valores en kW (razonables): 290
   ✓ Valores en pesos (> 10.000): 68
   ✓ Valores solo texto (sin número extraíble): 416

   Estadísticas de consumo_energia_kw_num (kW):
      - Mínimo: 0 kW
      - Máximo: 10,000 kW
      - Promedio: 1,346 kW
      - Mediana: 899 kW

   Estadísticas de consumo_energia_kw_cash (pesos):
      - Mínimo: $15,500
      - Máximo: $2,494,479,993
      - Promedio: $37,075,683
      - Mediana: $265,000



Podemos ver que el máximo de la variable `consumo_energia_kw_cash` es demasiado alto como para ser realista, analicemos que ocurrió con este registro:

In [13]:
df_clean[df_clean['consumo_energia_kw_cash'] > 500_000]['consumo_energia_kw_cash']

125      600000.0
136        600000
190      800000.0
201     2000000.0
209     1000000.0
247     1000000.0
262      692940.0
263      800000.0
318     1000000.0
354        900000
523      540000.0
544    2494479993
559        680000
565     1100000.0
650       1100000
820     1500000.0
Name: consumo_energia_kw_cash, dtype: object

Hay varios valores altos en los registros, pero conociendo la situación Argentina con respecto a las tarifas de electricidad estos pueden llegar a ser razonables y más para un negocio que necesita consumir electricidad constantemente. Sin embargo hay un caso en donde el número es demasiado alto como para ser realista ya que si lo cambiamos a dólares este representaría una cantidad absurda. Vamos a buscarlo por su índice en el conjunto de datos a ver que ocurrió:

In [14]:
print(df_clean.loc[544])

equipo                                                       Mit-Tandil Dara Hub
comercio                                                         La dulce espera
encuestador                                                      Delfina Sanchez
tipo_negocio                                                            Frutería
tiene_reja                                                                 False
anio_apertura                                                             2025.0
trabajadores_total                                                           2.0
trabajadores_salario_fijo                                                    0.0
expectativa_trabajadores_3m                                              Aumente
ventas_vs_mes_anterior                                                     Igual
razon_cambio_ventas            La gente administra sus ingresos de forma prec...
expectativa_ventas_3m                                                    Mayores
cambio_inventario           

Bueno en principio una frutería no debería pagar tanto por electricidad. Para más verificación, vamos a buscarla en el conjunto de datos original:

In [15]:

for columna in df_raw.loc[544].index:
    if columna == '¿Cuál fue el consumo energético del negocio el mes pasado en Kw?':
        print(f"{columna}: {df_raw.loc[544, columna]}")

¿Cuál fue el consumo energético del negocio el mes pasado en Kw?: 2494479993


Este es el valor en su estado crudo sin haber aplicado ningún preprocesamiento. No podemos inferir nada, no hay una coma o algo que nos diga que puede ser un número con decimales, así que antes de asumir un valor y potencialmente sesgar nuestro análisis lo mejor será hacerlo ver como valor faltante:

In [16]:
df_clean.loc[544, 'consumo_energia_kw_cash'] = None
print(df_clean.loc[544, 'consumo_energia_kw_cash'])

None


In [17]:
# Inspección de conversiones realizadas
print("="*80)
print("EJEMPLOS DE CONVERSIONES REALIZADAS")
print("="*80)

if 'salario_dejar_negocio_num' in df_clean.columns and 'salario_dejar_negocio_text' in df_clean.columns:
    print("\nSALARIO_DEJAR_NEGOCIO - Casos convertidos de texto a número:")
    print("-" * 60)
    
    # Filtrar registros que tenían texto y ahora tienen número
    mask = df_clean['salario_dejar_negocio_text'].notna() & df_clean['salario_dejar_negocio_num'].notna()
    ejemplos = df_clean[mask][['salario_dejar_negocio_text', 'salario_dejar_negocio_num']].head(15)
    
    if len(ejemplos) > 0:
        for idx, row in ejemplos.iterrows():
            print(f"   '{row['salario_dejar_negocio_text']}' → ${row['salario_dejar_negocio_num']:,.0f}")
    else:
        print("   (No hay casos con conversión exitosa)")
    
    print(f"\n   Casos con texto sin número extraíble:")
    solo_texto = df_clean[df_clean['salario_dejar_negocio_text'].notna() & df_clean['salario_dejar_negocio_num'].isna()]['salario_dejar_negocio_text'].head(10)
    for texto in solo_texto:
        print(f"   - '{texto}'")

if 'consumo_energia_kw_num' in df_clean.columns and 'consumo_energia_kw_text' in df_clean.columns:
    print("\n\nCONSUMO_ENERGIA_KW - Casos convertidos de texto a número (kW):")
    print("-" * 60)
    
    # Filtrar registros que tenían texto y ahora tienen número (kW razonables)
    mask = df_clean['consumo_energia_kw_text'].notna() & df_clean['consumo_energia_kw_num'].notna()
    ejemplos = df_clean[mask][['consumo_energia_kw_text', 'consumo_energia_kw_num']].head(15)
    
    if len(ejemplos) > 0:
        for idx, row in ejemplos.iterrows():
            print(f"   '{row['consumo_energia_kw_text']}' → {row['consumo_energia_kw_num']:,.0f} kW")
    else:
        print("   (No hay casos con conversión exitosa a kW)")
    
    # Mostrar casos identificados como dinero (pesos)
    if 'consumo_energia_kw_cash' in df_clean.columns:
        print(f"\n   Casos identificados como DINERO (pesos, no kW):")
        mask_cash = df_clean['consumo_energia_kw_cash'].notna()
        ejemplos_cash = df_clean[mask_cash][['consumo_energia_kw_text', 'consumo_energia_kw_cash']].head(15)
        
        if len(ejemplos_cash) > 0:
            for idx, row in ejemplos_cash.iterrows():
                texto = row['consumo_energia_kw_text'] if pd.notna(row['consumo_energia_kw_text']) else "N/A"
                print(f"   '{texto}' → ${row['consumo_energia_kw_cash']:,.0f}")
        else:
            print("   (No hay casos identificados como dinero)")
    
    print(f"\n   Casos con texto sin número extraíble:")
    solo_texto = df_clean[df_clean['consumo_energia_kw_text'].notna() & 
                          df_clean['consumo_energia_kw_num'].isna() & 
                          df_clean['consumo_energia_kw_cash'].isna()]['consumo_energia_kw_text'].head(10)
    for texto in solo_texto:
        print(f"   - '{texto}'")


EJEMPLOS DE CONVERSIONES REALIZADAS

SALARIO_DEJAR_NEGOCIO - Casos convertidos de texto a número:
------------------------------------------------------------
   '0 (no está dispuesto a venderlo)' → $0
   '$5000000' → $5,000,000
   '2 millones' → $2,000,000
   '$2500000' → $2,500,000
   '$1.000.000' → $1,000,000
   'lo dejarias por mayor flecibilidad o salario mas alto. 1500000' → $1,500,000
   '$600.000' → $600,000
   '$1.200.000' → $1,200,000
   '$1.200.000' → $1,200,000
   '2 millones' → $2,000,000
   '5 millones' → $5,000,000
   '$2000000' → $2,000,000
   '100000 USD' → $100,000
   '$2.000.000' → $2,000,000
   '1700000-2000000' → $1,700,000

   Casos con texto sin número extraíble:
   - 'no estaria dispuesta a dejarlo'
   - 'Por un aumento sustantivo de salario'
   - 'No aceptaria'
   - 'No está dispuesto a cerrar'
   - 'No aceptaría nada'
   - 'No lo acepta'
   - 'ninguna'
   - 'Ninguno'
   - 'Ninguna'
   - 'Ningúno'


CONSUMO_ENERGIA_KW - Casos convertidos de texto a número (kW):

In [18]:
df_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 923 entries, 0 to 922
Data columns (total 46 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   equipo                       905 non-null    object 
 1   comercio                     738 non-null    object 
 2   encuestador                  922 non-null    object 
 3   tipo_negocio                 923 non-null    object 
 4   tiene_reja                   923 non-null    bool   
 5   anio_apertura                853 non-null    float64
 6   trabajadores_total           853 non-null    float64
 7   trabajadores_salario_fijo    853 non-null    float64
 8   expectativa_trabajadores_3m  853 non-null    object 
 9   ventas_vs_mes_anterior       851 non-null    object 
 10  razon_cambio_ventas          703 non-null    object 
 11  expectativa_ventas_3m        851 non-null    object 
 12  cambio_inventario            848 non-null    object 
 13  expectativa_inventar

## 7. Resumen final y guardar datos limpios

In [19]:
# Versión limpia full
output_path = '../data/processed/clean_survey_full.csv'
df_clean.to_csv(output_path, index=False, encoding='utf-8')
print(f"\nDatos limpios guardados en: {output_path}")

columnas_eliminar = [
    'equipo', 'comercio', 'encuestador', 'whatsapp'
]
columnas_a_eliminar = [col for col in columnas_eliminar if col in df_clean.columns]
df_clean_to_analyze = df_clean.drop(columns=columnas_a_eliminar)

# Versión sin columnas no necesarias para el análisis
output_path = '../data/processed/clean_survey_to_analyze.csv'
df_clean_to_analyze.to_csv(output_path, index=False, encoding='utf-8')
print(f"\nDatos limpios guardados en: {output_path}")


Datos limpios guardados en: ../data/processed/clean_survey_full.csv

Datos limpios guardados en: ../data/processed/clean_survey_to_analyze.csv
